<a href="https://colab.research.google.com/github/tucaoneo/demo/blob/master/Istanbul.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Istanbul is a boardgame where players (merchants) compete to be the most successful ruby collector in the capital of the Ottoman Empire

https://boardgamegeek.com/boardgame/148949/istanbul

This code is to solve certain probability problems in the game



In [1]:
import pandas as pd
import numpy as np

1. Tea House

1.1. General Rule: Announce a number between 3 and 12. Then roll both dice. If you roll equal to or greater than the announced number, take the announced number of Lira from the general supply. Otherwise, only take 2 Lira.

Announced number is in [3, 12]. It's interesting that announcing 2 is forbidden. Let lira[x] be the expected gain from announcing x, where x = 3, ..., 12.

Actual result (the sum of both dice) is in [2, 12]. Let p[y] = the probability that the result is y, where y = 2, ..., 12.

In [2]:
p = [0, 0] + [i / 36 for i in range(1, 7)] + [i / 36 for i in range(5, 0, -1)]
p

[0,
 0,
 0.027777777777777776,
 0.05555555555555555,
 0.08333333333333333,
 0.1111111111111111,
 0.1388888888888889,
 0.16666666666666666,
 0.1388888888888889,
 0.1111111111111111,
 0.08333333333333333,
 0.05555555555555555,
 0.027777777777777776]

In [3]:
lira = [0, 0, 0]
for x in range(3, 13):
    lira.append(sum(p[x:]) * x + sum(p[:x]) * 2)
lira

[0,
 0,
 0,
 2.9722222222222223,
 3.8333333333333335,
 4.500000000000001,
 4.888888888888889,
 4.916666666666667,
 4.5,
 3.944444444444444,
 3.333333333333334,
 2.7500000000000004,
 2.277777777777778]

lira[x] is maximized at x = 7. So you should announce 7. The maximum expected gain 4.917.

1.2. Red Tile Ability: If you have the red Mosque tile, you may turn one
die to “4” after the roll or re-roll both dice. For simplicity, we will say we turn or re-roll.

Let lessThan[z] be the probability that the result of one die is smaller than z, where z = 1, ..., 6.

Let liraRedTile[x] be the expected gain from announcing x, now that we are armed with the red tile ability, where x = 3, ..., 12.

The red tile allows you to be somewhat bolder when announcing numbers. There are three situations based on what you announce: 

Suppose you are a cautious player and you announce a number x <= 5. Now you are guaranteed to gain x, because you can always get result >= x by turning. Therefore liraRedTile[x] = x for x = 3, 4, 5.

Suppose you announce x = 6, ..., 10. If the first roll fails, you will decide whether to turn or re-roll, depending on how the two dice are.

Finally, suppose you are very aggressive and announce x >= 11. Now, if your first roll fails, turning will not help you. You will have to re-roll hope you get lucky.

In [4]:
lessThan = [0] + [(z - 1) / 6 for z in range(1, 7)]
lessThan

[0,
 0.0,
 0.16666666666666666,
 0.3333333333333333,
 0.5,
 0.6666666666666666,
 0.8333333333333334]

In [5]:
liraRedTile = [0, 0, 0, 3, 4, 5]
for x in range(6, 11):
    liraRedTile.append((1 - lessThan[x - 4] ** 2 + lessThan[x - 4] ** 2 * sum(p[x:])) * x\
                       + lessThan[x - 4] ** 2 * sum(p[:x]) * 2)
for x in range(11, 13):
    liraRedTile.append((sum(p[x:]) + sum(p[:x]) * sum(p[x:])) * x\
                       + (sum(p[:x]) - sum(p[:x]) * sum(p[x:])) * 2)
liraRedTile

[0,
 0,
 0,
 3,
 4,
 5,
 5.969135802469135,
 6.768518518518518,
 7.125,
 6.753086419753087,
 5.370370370370369,
 3.4375,
 2.5478395061728403]

liraRedTile[x] is maximized at x = 8. So you should announce 8. We can see the red tile has significantly boosted the expected maximum gain from 4.917 to 7.125.

Put all results in a data frame

In [6]:
TH_ExpectedGains = pd.DataFrame({'AnnouncedNumber': list(range(13)), 'Original': lira, 'WithRedTile': liraRedTile})
TH_ExpectedGains["RedTileBoost"] = TH_ExpectedGains["WithRedTile"] - TH_ExpectedGains["Original"]
TH_ExpectedGains

,AnnouncedNumber,Original,WithRedTile,RedTileBoost
0,0,0.000000,0.000000,0.000000
1,1,0.000000,0.000000,0.000000
2,2,0.000000,0.000000,0.000000
3,3,2.972222,3.000000,0.027778
4,4,3.833333,4.000000,0.166667
5,5,4.500000,5.000000,0.500000
6,6,4.888889,5.969136,1.080247
7,7,4.916667,6.768519,1.851852
8,8,4.500000,7.125000,2.625000
9,9,3.944444,6.753086,2.808642


2. Black Market

2.1. General Rule (the part that involves rolling dice):
Roll both dice.
If you rolled 7 or 8: Gain 1 blue good.
If you rolled 9 or 10: Gain 2 blue goods.
If you rolled 11 or 12: Gain 3 blue goods.

Don't forget the limited room on your wheelbarrow. If your wheelbarrow doesn't have enough room for all the blue goods you can gain, then you load up your wheelbarrow and lose the extra blue goods.

Let blue[y] be the number of blue goods to gain if the rolled result is y, where y = 2, ..., 12.

Let expectedBlue[w] be the expected number of blue goods to gain if there are w empty slots on the wheelbarrow, where w = 0, ..., 5.

In [7]:
blue = [0] * 7 + [1] * 2 + [2] * 2 + [3] * 2
blue

[0, 0, 0, 0, 0, 0, 0, 1, 1, 2, 2, 3, 3]

In [8]:
expectedBlue = [0]
for w in range(1, 6):
    gain = 0
    for y in range(2, 13):
        gain += p[y] * min(w, blue[y])
    expectedBlue.append(gain)
expectedBlue

[0,
 0.5833333333333334,
 0.8611111111111112,
 0.9444444444444444,
 0.9444444444444444,
 0.9444444444444444]

So if you have 1 empty slot on your wheelbarrow, you expect to gain 0.58 blue good. If you have 2 empty slots, you expect to gain 0.86 blue good. If you have 3 or more empty slots, you expect to gain 0.94 blue good.

2.2. Red Tile Ability: If you have the red Mosque tile, you may turn one
die to “4” after the roll or re-roll both dice.

To analyze the situation, one needs to break it down.

If y = 2, 3, turning will not help. You have to re-roll.

If y = 4 and it's 3 + 1, turning will make sure you gain 1 blue good instead of 0. Remember that, one roll without special ability is expected to gain less than 1 blue good. So you don't want to re-roll here.

In fact, as long as you gain at least 1 blue good, re-rolling is never a good thing to do. In the situations below, you will never re-roll.

If y = 4 and it's 2 + 2, then you gain nothing, even if you turn. You have to re-roll.

If y = 5, turning will make sure you gain 1 blue good instead of 0.

If y = 6 and it's 5 + 1, turning will make sure you gain 2 blue goods instead of 0.

If y = 6 and it's 4 + 2 or 3 + 3, turning will make sure you gain 1 blue good instead of 0.

If y = 7 and it's 6 + 1 or 5 + 2, turning will make sure you gain 2 blue goods instead of 1.

If y = 7 and it's 4 + 3, you will gain 1 blue good, and turning will make no difference.

If y = 8 and it's 6 + 2 or 5 + 3, turning will make sure you gain 2 blue goods instead of 1.

If y = 8 and it's 4 + 4, you will gain 1 blue good, and turning will not make a difference.

If y = 9, 10, you gain 2 blue goods, and turning will not help.

If y = 11, 12, you gain 3 blue goods.

With all that, let expectedBlueRedTile[w] be the expected number of blue goods to gain if there are w empty slots on the wheelbarrow, where w = 0, ..., 5.

In [9]:
expectedBlueRedTile = [0]
for w in range(1, 6):
    gain = 0
    for y in range(2, 4):
        gain += p[y] * expectedBlue[w]
    # y = 4
    gain += 2 / 36 * min(w, 1)
    # y = 5
    gain += p[5] * min(w, 1)
    # y = 6
    gain += 2 / 36 * min(w, 2) + 3 / 36 * min(w, 1)
    # y = 7
    gain += 4 / 36 * min(w, 2) + 2 / 36 * min(w, 1)
    # y = 8
    gain += 4 / 36 * min(w, 2) + 1 / 36 * min(w, 1)
    for y in range(9, 13):
        gain += p[y] * min(w, blue[y])
    expectedBlueRedTile.append(gain)
expectedBlueRedTile

[0,
 0.9375,
 1.5162037037037037,
 1.6064814814814816,
 1.6064814814814816,
 1.6064814814814816]

Put all results in a data frame

In [10]:
BM_ExpectedGains = pd.DataFrame({'WheelbarrowEmptySlots': list(range(6)), 'Original': expectedBlue, 'WithRedTile': expectedBlueRedTile})
BM_ExpectedGains["RedTileBoost"] = BM_ExpectedGains["WithRedTile"] - BM_ExpectedGains["Original"]
BM_ExpectedGains

,WheelbarrowEmptySlots,Original,WithRedTile,RedTileBoost
0,0,0.000000,0.000000,0.000000
1,1,0.583333,0.937500,0.354167
2,2,0.861111,1.516204,0.655093
3,3,0.944444,1.606481,0.662037
4,4,0.944444,1.606481,0.662037
5,5,0.944444,1.606481,0.662037
